# Karthik Gururangan
# EECS 475/495 Final Project
### ** Source code and dataset available at https://github.com/kgururangan/Redfield_GPR

## 1. Modeling Exciton Dynamics with Redfield Theory
A major goal of chemical physics is to unravell the non-trivial relaxation processes that occur in a stimulated quantum mechanical system interacting with its environment (called an open system). Such questions arise in a variety of contexts, for instance, following photo-extcitation in optical/IR/THz or NMR spectroscopies. The primary object of concern in all questions of non-equilibrium quantum dynamics is the time evolution of the total-system density matrix $\rho(t)$ with elements defined within a suitable basis $\{ |a\rangle, |b\rangle, |c\rangle, \ldots \}$ as $\rho_{ab}(t)\equiv \langle a| \rho (t)| b \rangle$. Formally, the time evolution is governed by the Liouville-von-Neumann equation:

$$\frac{d \rho}{d t} = [\hat{H},\rho(t)] = \hat{H}\rho - \rho\hat{H}$$

where $\hat{H}$ is the total system Hamiltonian. Generally, in condensed-phase systems, one is interested in the response of a small part of a system (called the "system") embedded in a larger one (called the "bath"). For instance, in condensed-phase spectroscopy, the experimental signal measures the response of a solute within a solvent. To incorporate this in the equations, the total system is partitioned into a system, bath, and system-bath interaction: $\hat{H} = \hat{H}_s + \hat{H}_b + \hat{H}_{sb}$. With this partitioning, we wish to examine the behavior of the reduced system density matrix $\sigma(t) = \mathrm{Tr}_b(\rho(t))$ under a coarse-grained action due to the system-bath interaction. A proper discussion of this topic is well outside of the scope of this work, and it will suffice to say that, in general, the reduced system density matrix $\sigma(t)$ can be shown to follow a non-Markovian stochastic differential equation

$$\dfrac{d\sigma}{dt} = \mathcal{L}_s \sigma + \int_0^t d\tau G(t-\tau)\sigma(\tau)$$

where $G(t-\tau)$ is the non-Markovian memory kernel capturing the effects of coarse-graining on $\sigma$ and $\mathcal{L}_s$ is the Liouville operator of the system. Under the assumptions of weak system-bath coupling and the Markov approximation, the kernel can be evaluated independently of the state of the bath and the integro-differential equation reduces to a simple rate law expression

$$\dfrac{d\sigma_{ab}}{dt} = \sum_{cd} R_{abcd}\sigma_{cd} \Rightarrow \dfrac{d \vec{\sigma}}{dt} = \textbf{R}\vec{\sigma}$$

where the object $R_{abcd}$ is a 4$^\mathrm{th}$ rank tensor called the Redfield tensor. We typically vectorize the density matrix to make the tensor problem amenable to standard matrix calculations. The elements of the Redfield tensor are related to Fourier transforms of the bath correlation functions: $R_{abcd} \sim \int_{-\infty}^{\infty} d\tau e^{-i\omega \tau}\langle V_{ab}(\tau)V_{cd}(0) \rangle$. Many schemes exist for calculating the Redfield tensor and we have chosen an implementation detailed in Ref. [1] called Coherent Modififed Redfield Theory (CMRT). The Redfield tensor is a function of several variables relevant to molecular systems. The ones we will focus on in this work are the dimensionless displacements $(d_i)$ (or Huang-Rhys factors) characterizing the exciton-phonon coupling of a single chromophore to its vibrational manifold. A major problem with Redfield and related quantum dynamic theories is that the calculation of the tensor can be a computationally expensive process for multilevel systems. However, a working model detailing the dependence of transfer rates as a function of various relevant microscopic parameters is of great use for interpretation of experimental data or design purposes. In fact, solving this all-too common problem in physical sciences leads to a practice called "metamodeling" - building a model of the model (Ref. [2]). For this reason, we will explore using Gaussian process regression to construct a metamodel for qualitative or quantitative predictions of exciton transfer kinetics. 

We investigate a two-level sytem with one vibrational mode. In total, this enumerates four states $\{ |0\rangle_1, |1\rangle_0, |0\rangle _1, |1\rangle _1\}$ with $|\rangle_0$ and $|\rangle _1$ denoting the first and second manifolds, respectively. For a dimer system examined in this work, there exist only two displacements $d_1$ and $d_2$ that constitute the feature space $\textbf{x}_p$. To frame the problem in an ML context, we require a simple (ideally 1-dimensional) figure of merit assigned to every point in the feature space. Following a previous work on this subject (Ref. [3]), we will use the nominal population growth rate of the lowest-energy state $|0\rangle _0$ (characterized by the time evolution of the density matrix element $\sigma_{11}(t)$) and assign this aggregate transfer rate $k_{\mathrm{Tr}}$ as the output $y_p$ corresponding to every point in the feature space $\textbf{x}_p = (d_1,d_2)$.

To summarize: $$R_{abcd} = R_{abcd}(d_1,d_2) \Rightarrow \textbf{R}(\textbf{x}_p)$$
$$\dfrac{d\sigma_{11}}{dt} = \sum_{ij} R_{11ij}\sigma_{ij}$$ Generally, the time evolution of $\sigma_{11}(t)$ is well-approximated by a first-order rate law expression and can be fit to the curve $$\mathrm{model}(t,c_0,c_1,k_{\mathrm{Tr}}) = c_0 + c_1 (1 - e^{-k_{\mathrm{Tr}}t})$$
where $k_{\mathrm{Tr}}$ is the nominal transfer time, $c_0$ is the population at $t\rightarrow 0$ and $c_1$ is the population at $t \rightarrow \infty$. Therefore, $k_{\mathrm{Tr}}$ is the figure of merit ($y_p$) we seek.

The dataset was generated using a homemade code implementing CMRT. The code was written in both Python and Matlab, but the Matlab version proved to give faster results. The calculation of $\textbf{R}$ for each point in the design space took approximately 1 minute. A $40\times 40$ grid of $(d_1,d_2)$ was sampled to create the input dataset and the calculation took approximately 1.5 days to run. The tasks relevant to this work are the following:

(1) Given an input of $P=1600$ different Redfield tensors $\{\textbf{R}_p\}_{p=1}^{P=1600}$ corresponding to each $\textbf{x}_p = (d_1,d_2)_p$, the differential equation for $\sigma(t)$ was integrated and the resulting curve was fit to the rate law model to find each $y_p$. This step effectively generates the dataset $\{\textbf{x}_p,y_p\}_{p=1}^{P=1600}$

(2) Using Gaussian process regression to optimally interpolate this dataset, a predictive metamodel of the transfer efficiency was created for future design and optimization purposes
 
(3) Model evaluation using cross-validation

## 2. Gaussian Process Regression

In this section, we will introduce the concept of a Gaussian process regression (GPR) and give a derivation of the GPR equations using the technique of kernel functions. Explicit parameterization of the mean, often called the Kriging method, is also introduced. This discussion is largely drawn from Ref. [4] and only scratches the surface of GPs. Gaussian processes have a rich mathematical structure which can be linked to many ML problems including classification, neural networks, and Bayesian optimization (see Ref. [5]). 

### 2.1 Theoretical Concept for Linear Models
The likelihood of a given model $\textbf{w}$ is defined by $L(\textbf{w}) = P(D|\textbf{w}) = \prod_{p=1}^P P(y_p|\textbf{x}_p, \textbf{w})$ where each individual data point is assumed to be independent and Gaussian distributed: $P(y_p|\textbf{x}_p,\textbf{w}) \sim \mathcal{N}(\textbf{w}^T \textbf{x}_p,\sigma_p^2)$. Given a suitable prior distribution over models $P(\textbf{w})$, we can express the posterior distribution using Bayes' Rule:
$P(\textbf{w}|D) = L(\textbf{w})P(\textbf{w})(P(D))^{-1} = Z^{-1}L(\textbf{w})P(\textbf{w})$ where $Z = P(D)$ can be taken as a normalization constant. In general, we will further assume the prior to be Gaussian distributed $P(\textbf{w}) \sim \mathcal{N}(0,\Sigma_w)$

Recall the definition of a Gaussian distribution with mean $\textbf{x}_0$ and covariance $\Sigma$ is $$\mathcal{N}(\textbf{x}_0,\Sigma) = (2\pi |\Sigma|)^{1/2}\exp\left[-\frac{1}{2}(\textbf{x}-\textbf{x}_0)^T\Sigma^{-1}(\textbf{x}-\textbf{x}_0)\right]$$
where $|\Sigma| \equiv \mathrm{det}(\Sigma)$. Working out the form of the posterior distribution, we have:
$$P(\textbf{w}|D) \propto \prod_{p=1}^P \mathcal{N}(\textbf{w}^T\textbf{x}_p,\sigma_p^2) \mathcal{N}(\textbf{0},\Sigma_w)\propto \exp\left[ -\frac{1}{2}\sum_p \left(\sigma_p^{-2}(y_p - \textbf{x}_p^T\textbf{w})^T(y_p-\textbf{x}_p^T\textbf{w}) + \textbf{w}^T\Sigma_w\textbf{w}\right)\right]$$

$$\propto \exp\left[ -\frac{1}{2}\left(\textbf{w}^T\Sigma_w\textbf{w} + \frac{1}{\sigma_p^2\textbf{I}}(\textbf{X}^T\textbf{w}\textbf{w}^T\textbf{X} - (\textbf{y}^T\textbf{w}^T\textbf{x} + \textbf{x}^T\textbf{w}\textbf{y})\right) \right] \propto \exp\left[ -\frac{1}{2}(\textbf{w}-\textbf{w}_0)^T\textbf{A}(\textbf{w}-\textbf{w}_0) \right]$$


Where $\textbf{w}_0 = [\frac{1}{\sigma_p^2\textbf{I}}(\textbf{X}\textbf{X}^T + \Sigma_w^{-1})]^{-1}
\textbf{y}$ and $\textbf{A} = \frac{1}{\sigma_p^2\textbf{I}}\textbf{X}\textbf{X}^T + \Sigma_w^{-1}$ are defined by completing the square in the exponential. Furthermore, we have vectorized the output $\textbf{y} = [y_1\ldots y_p]^T$ and correspondingly defined $\textbf{X} = \left[ \textbf{x}_1, \textbf{x}_2, \ldots, \textbf{x}_P \right]$ as the design matrix (to avoid carrying around the $\sum_p$ in the exponential). $\textbf{C} = (\sigma_p\textbf{I})^{-2}\textbf{X}\textbf{X}^T$ is the covariance matrix of the data and $\textbf{A}$ is the covariance of the posterior distribution. From this analysis, we conclude that the posterior is also a Gaussian distribution:

$$P(\textbf{w}|D) \sim \mathcal{N}(\frac{1}{\sigma_p^2\textbf{I}}\textbf{A}^{-1}\textbf{X}\textbf{y},\textbf{A}^{-1})$$

Note that for $\Sigma_w = 0$, this is simply the solution to the normal equations for least-squares regression ($\textbf{w} = (\textbf{X}\textbf{X}^T)^{-1}\textbf{X}\textbf{y})$. The Gaussian nature of the posterior is a direct consequence of choosing a Gaussian prior $P(\textbf{w})$. Consider a set of test points $\{\textbf{x}_*\}_{n=1}^{n_S}$ for which we would like to make predictions $\{y_*\}_{n=1}^{n_S}$. If this is our objective, we can in fact write an expression that accomplishes this goal independent of the assumed model. Namely, this is accomplished by integrating over all models:
$$P(y_*|\textbf{x}_*,D) = \int_{\textbf{w}}P(y_*|\textbf{x}_*,D,\textbf{w}) d\textbf{w} = \int_{\textbf{w}}P(y_*|\textbf{x}_*,\textbf{w})P(\textbf{w}|D) d\textbf{w}$$

Plugging in the form of $P(\textbf{w}|D)$ found above, we can repeat the same process of collecting terms and completing the square to find that the prediction $y_*$ at the test point $\textbf{x}_*$ is also a Gaussian distribution with mean and covariance given by

$$P(y_*|\textbf{x}_*,D) \sim \mathcal{N}(\frac{1}{\sigma_p^2\textbf{I}}\textbf{x}_*^T\textbf{A}^{-1}\textbf{X}\textbf{y}, \textbf{x}_*^T\textbf{A}^{-1}\textbf{x}_*)$$

(Another way to quickly arrive to this result is to notice that, using the least-square regression perspective, $\mathrm{model}(\textbf{w},\textbf{x}) = \textbf{x}^T\textbf{w}$ where $\textbf{w} = \textbf{A}^{-1}\textbf{X}\textbf{y}$, so $y_* = \textbf{x}_*^T\textbf{w}$ produces the mean of the predictive distribution).

###  2.2 Extension to Nonlinear Models using the Kernel Trick

Now assume that instead of a linear model, we use a collection of $B$ feature transformations $\pmb{\phi}\equiv\{\phi_i(\textbf{x})\}_{i=1}^B$ to write our nonlinar model as $\mathrm{model}(\textbf{x},\pmb{\theta}) = \pmb{\phi}^T\textbf{w}$ where the collection of hyperparameters $\pmb{\theta} = [\textbf{w}, \textbf{v}]$ where $\textbf{v}$ denotes all internal parameters of the feature transformations. With these definitions, the design matrix becomes a matrix $\pmb{\Phi}$ with elements $\Phi_{ij} = \phi_i(\textbf{x}_j)$, but the form of the predictive distribution remains the same:
$$P(y_*|\textbf{x}_*,D) \sim \mathcal{N}(\mu_*, \Sigma_*) = \mathcal{N}(\frac{1}{\sigma_p^2\textbf{I}}\pmb{\phi}_*^T\textbf{A}^{-1}\pmb{\Phi}\textbf{y}, \pmb{\phi}_*^T\textbf{A}^{-1}\pmb{\phi}_*)$$
where $\textbf{A} = \frac{1}{\sigma_n^2\textbf{I}}[\pmb{\Phi}\pmb{\Phi}^T + \Sigma_w^{-1}]$. Let us consider the mean and covariance 
$$\mu_* = (\sigma_p\textbf{I})^{-2}\pmb{\Phi}_*^T(\pmb{\Phi}\pmb{\Phi}^T + \Sigma_w)^{-1}\pmb{\Phi}\textbf{y} = (\sigma_p\textbf{I})^{-2}\Sigma_w$$
$$\Sigma_* = \pmb{\Phi}_*^T(\pmb{\Phi}\pmb{\Phi}^T+\Sigma_w)^{-1}$$

By performing algebraic manipulations that can be seen in detail in Ref. [4], this expression can be rewritten in a compact form using only combinations of $\pmb{\Phi}_{(*)}\Sigma_w\pmb{\Phi}_{(*)}^T$ - this is referred to as the "kernel trick" and generates the central equations of GPR:

$$\mu_* = \textbf{K}_*^T(\textbf{K} + \sigma_p^2 \textbf{I})^{-1}\textbf{y} \\
\Sigma_* = \textbf{K}_{**} - \textbf{K}_*^T(\textbf{K} + \sigma_p^2\textbf{I})^{-1}\textbf{K}$$

where we have defined the kernel matrix using a block representation
$$(\textbf{K})_{\mathrm{Total}} =\begin{bmatrix} K(\textbf{X},\textbf{X}) & K(\textbf{X},\textbf{X}_*) \\ K(\textbf{X}_*,\textbf{X}) & K(\textbf{X}_*,\textbf{X}_*) \end{bmatrix} = \begin{bmatrix} \textbf{K} & \textbf{K}_* \\ \textbf{K}_*^T & \textbf{K}_{**} \end{bmatrix}$$

The mean $\mu_*$ is the predictor of the kernel regression (i.e. the function we will use to build the metamodel) and $\Sigma_*$ is a function expressing the covariance for any point in the design space (error estimate of the prediction). The kernel function $K(x,z)$ can take on various parameterizations based on the choice of basis function $\{\phi_i\}$, such as polynomials or Fourier components. The kernel used in this work is the Gaussian kernel defined by
$$K(x,z) = \exp\left(\dfrac{(x-z)^2}{\theta^2}\right)$$ where $\theta$ is the length scale hyperparameter. This choice of kernel is very popular and corresponds to a basis set of $\phi_b(x) = e^{-\theta^{-1}x^2}\left(\frac{(2\theta^{-1})^{b-1}}{(b-1)!}\right) x^{b-1}$. In most cases, the actual basis functions are of less use for understanding the kernel method than the kernel itself. The kernel function $K(x,z)$ is a model for the spatial correlation function in the design space. Most kernels use a quite intuitive model that points close to each other have correlated responses while those far away are independent. The length scale hyperparameter $\theta$ controls the length scale of correlations. For larger values of $\theta$, points are correlated over a longer distance and vice-versa. Another important caveat is the factor of $\sigma_p^2\textbf{I}$ added to the test matrix kernel $\textbf{K}$. Recall that $\sigma_p^2$ expresses the variance associated with the $p^{\mathrm{th}}$ data point. In general for computer simulations, $\sigma_p^2 = 0$; however, in many situations (and indeed in the calculations in this work), $\textbf{K}$ can becomes semi-definite, rendering its inversion singular. Therefore, when using noiseless data, the factor $\sigma_p^2 \textbf{I} \rightarrow \epsilon \textbf{I}$ is retained under the guise of a small regularization parameter $\epsilon \sim 10^{-9} - 10^{-6}$ to stabilize the inversion. In this work, it was found that $\textbf{K}$ was highly prone to this instability when the training set became very dense over the design space. In these cases, increasing $\epsilon$ to as high as $10^{-6}$ was necessary. Another computational addendum related to this point is the use of the Cholesky decomposition of $(\textbf{K}+\epsilon\textbf{I}) = \textbf{L}\textbf{L}^T$ for added efficiency and numerical stability. The Cholesky decomposition is only valid for symmetric (Hermitian) positive-definite matrices and still requires the regularization parameter $\epsilon$ for when $\textbf{K}$ becomes semi-definite. 

### 2.2 Explicit Parameterization of the Mean
The above description of GPR assumes no modes for the mean or covariance. While this is perhaps the most powerful aspect of GPR, in many physical science modeling scenarios, we have some idea (or visual belief) that there is a global trend of the mean. In these cases, we explicitly parameterize 

$$\mu_* = \sum_i \beta_i \Psi_i = \textbf{F}\pmb{\beta}$$

where $\Psi_i$ are a collection of some low-order polynomials and $[\textbf{F}]_{ij} = \Psi_i(\textbf{x}_j)$ is the model-transformed design matrix. This explicit parameterization of the mean is often called the Kriging method. This situation is very common in response surface modeling where each $\Psi_i$ corresponds to a particular design variable $x_i$. Cross-terms such as $x_i x_j$ imply some degree of correlation between different variables. In this case, the GPR equations are modified to solve for the polynomial coefficients:

$$\pmb{\beta} = (\textbf{F}_*^T\textbf{K}\textbf{F})^{-1}\textbf{F}_*^T\textbf{K}^{-1}\textbf{y}$$
$$\Sigma_* = (\textbf{y}-\textbf{F}\pmb{\beta})^T\textbf{K}^{-1}(\textbf{y}-\textbf{F}\pmb{\beta})$$

The Kriging predictor used to query any point in the design space is 
$$\mu_* \equiv \mu(\textbf{x}_*) = \textbf{f}(\textbf{x}_*)^T\pmb{\beta} + \textbf{k}^T(\textbf{x}_*)\textbf{K}^{-1}(\textbf{y}-\textbf{F}\pmb{\beta})$$
where in all of these expression, $\textbf{K}\equiv (\textbf{K}+\epsilon\textbf{I})$ is implied for stability. When the mean is unparameterized, $\Psi_i = 1$ and the Kriging predictor reduces to the original GPR equations in Sect. 2.1. In this code, we have implemented polynomial parameterization up to 2nd order, meaning that 
$$\mu = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_{11}x_1^2 + \beta_{22}x_2^2 + \beta_{12}x_1 x_2$$ 
In this work, we have denoted 4 types of models denoted by a degree $d$ that can be used up to order 2. The constant mean uses only $\beta_0$ and is called $d = 0$. When $\beta_0$, $\beta_1$, and $\beta_2$ are retained, $d =1$, when $\beta_0$, $\beta_1$, $\beta_2$, $\beta_{11}$, and $\beta_{22}$ are retained, $d=2$, and when all terms including $\beta_{12}$ are retained, $d = 2.5$. The significance between $d=2$ vs. $d=2.5$ is the explicit parameterization of interaction between design variables. 


### 2.3 Choosing the Hyperparameters 
The final aspect of using GPR is deciding on the choice of hyperparameter $\theta$. A full discussion is given in Ref. [4]. The simplest way to achieve this task is by maximizing the marginal likelihood estimate of the Kriging model given by

$$L(\theta) = \dfrac{1}{(2\pi |\Sigma_*|)^{N/2}|\textbf{K}|^{1/2}}\exp\left(\dfrac{1}{2|\Sigma_*|}(\textbf{y}-\textbf{F}\beta)^T\textbf{K}^{-1}(\textbf{y}-\textbf{F}\beta)\right)$$

$$\min_{\theta} [-\log(L(\theta))]$$

Since $\textbf{K}$ is a function of $\theta$, both $\pmb{\beta}$ and $\Sigma$ are implicitly functions of $\theta$. The equations of Section 2.2 must be solved for each evaluation of $L(\theta)$. Although log-likelihoods are known to be non-convex and somewhat tricky functions to optimize, the task is efficiently carried out using momentum-accelerated gradient descent. For the Kriging method, there exists another hyperparameter in our model, namely, the polynomial degree $d$. This values is most efficiently set by trying out all orders of models and finding the best one using cross-validation. In this work, we have performed cross-validation by choosing a random training set of the data to built the metamodel, then validating the choice of $d$ by the mean-square error of the prediction against all points not in the training set. 

## 3. Examples of GPR in Action

In ordinary regression, we are forced to pick a functional form of the model and optimize the parameters so as to minimize the least-squares cost deviation from the data. The incredible thing about GPR is that it can be performed without any model selection. The only parameterization we introduce comes in the form of the kernel function $K(\textbf{x},\textbf{z})$, however, the kernel is not describing the functional form of the data itself, rather, it is only a model for how correlated we believe adjacent data points are with one another. This is a key distinction - correlations are something much more universal and one kernel can accurately describe a huge variety of funcitonal forms. Furthermore, thanks to the kernel trick, we can subsume the huge amount of complexity associated with writing the model $\textbf{w} = \sum_i \alpha_i \phi_i(\textbf{x})$, where each $\phi_i(\textbf{x})$ could depend on a number of parameters, into the few hyperparameters of $K(\textbf{x},\textbf{z})$. Last, and certainly not least, GPR gives us error estimates on every prediction practically for free, the usefulness of which cannot be overstated. Its potency is best examined through some examples. 

Consider the following oscillatory function (actually the 50$^{\mathrm{th}}$ eigenfunction of the quantum harmonic oscillator $\psi_{50}(x)\propto H_{50}(x)e^{-x^2/2}$). It's a rather complicated-looking function, requiring either substantial familiarity with wave mechanics to identify a single feature transformation, or more likely, use of a Fourier series approximation. However, notice that, due to the Gaussian envelope, the function dies rapidly for $|x| > 2$. Capturing this behavior would require a huge number of Fourier components to adequately fit this function. But note that these issues of fititng the function exist even when the function is handed to us with perfect resolution; in this example, we are only supplied with 22 training points to fit this curve!

<img src="GPR_1D_Example_data.svg">
<img src="GPR_1D_Example_hermite.svg">
<img src="GPR_1D_Example_2.svg">

First, we optimize the hyperparameter $\theta$ to set the proper correlation length scale. This is done my maximizing the marginal log-likelihood, as described in Section 2.4. Note that the cost histories imply that several of the runs of gradient descent got trapped in local minima. This is a common feature of maximizing log-likelihood functions and care must be taken to run the optimization using several starting points. In more complex scenarios, Monte Carlo or simulated annealing approaches are used to ensure convergence to a global minimum at the expense of runtime (Ref. [2]). Using GPR, we get a reasonable estimate for the curve, and most importantly, we get error estimates for every point. Clearly, the true curve lies within the error bounds set by GPR. This all happens without giving the algorithm a model; in this example, the polynomial degree is set to $0$ so this is truly model-independent regression. Also note that GPR is interpolating through the supplied training set and when it passes through each point, the error goes to 0. The form of the error bars reflect both the choice of kernel and length scale parameter. For larger values of $\theta$, the error bars try to keep the same value over a longer distance while for smaller $\theta$, they are subject to more rapid change. This is why optimizing the hyperparameter using the log-likelihood is a such an important part of properly using GPR.   

GPR can be used in any number of dimensions and we will show its ability to fit a 2D surface as a function of the number of training points. For brevity, we have omit showing the log-likelihood cost function history curves here.

<img src="GPR_2D_Example.png">

In these images, the red surfaces indicate the $\pm 2\sigma$ error bars. Clearly, it struggles with incredibly sparse sampling and fails to really fit the structure of the surface when trained on 2-5% of the data. However, with only 10%, it actually fits the surface quite well. By the time it's training on >20%, it has the entire surface fit almost exactly. We also note that the choice of traning points will play a large role on how well the surface fits. In all calculations in this work, random sampling of the data was used to generate the training set. This is a $0^{\mathrm{th}}$-order way of creating training data. If the objective is to create a high-fidelity interpolation over a large design space, it is obvious one should pick points that span the space in some optimal sense. There are many optimality criteria, including maximization of entropy, maximization of minimum distance (MAXIMIN), centered-$\mathrm{L}_2$ discrepancy, and orthogonal latin hypercubes. For more details, see Ref. [6].

## 4. Metamodel Construction for Redfield Data
The construction of a Kriging metamodel for the dataset of Redfield theory calculations follows exactly as the examples shown in Section 3. First, the data was loaded in to provide the tensors $\{\textbf{R}\}_{p=1}^{P=1600}$. The equation of motion $$\dfrac{d\vec{\sigma}}{dt} = \textbf{R}\vec{\sigma}$$ was integrated using a written $4^{\mathrm{th}}$-order Runge-Kutta (RK4) integrator to obtain the following time evolution results for the denstiy matrix:

<img src="Redfield_GPR_RHO.png">

The initial condition for all results uses $\vec{\sigma}(0) = [0, 0, 0.5, 0.5]^T$ which is a very reasonable assumption for the population distribution of a system of chromophores following photo-excitation. Next, each of the curves showing the evolution of $\sigma_{11}(t)$ (called $\rho_{11}(t)$ in the figure) was fit to the exponential growth model 

$$\mathrm{model}(t,c_0,c_1,k_{\mathrm{Tr}}) = c_0 + c_1(1 - e^{-k_{\mathrm{Tr}}t})$$ 

to obtain $y_p$ as explained in Section 1. The resulting fitted growth rates are shown in the following contour plot:

<img src="Redfield_GPR_RateContour.png">

This fitting was accomplished using the scipy.optimize package and the "curvefit" method implementing the Levenberg-Marquardt algorithm. This was found to perform much better than fitting using the gradient descent algorithms used in this class due to the Levenberg-Marquardt's larger tolerance on the closeness of the initial point. Now that we have a simple 2D surface representation of the data, we can perform GPR as outlined in Section 3 using the four different polynomial models characterized by $d = 0, 1, 2, 2.5$. The GPR results for each are shown below along with cost histories for the log-likelihood optimization to set the length scale hyperparameter. 

<img src="Redfield_GPR_Results_6.png">
<img src="Redfield_GPR_Hyperrparam_Results_2.svg">
<img src="Redfield_GPR_CV_Results_2.svg">

Finally, cross-validation was performed to check which model gives the best predictive results. From the CV scores, it is clear that $d = 2.5$ performs the best, indicating that accounting for correlation between the vibronic coupling of each manifold ($d_1$ and $d_2$) is an important factor in getting the right predictions. Furthermore, it is interesting to note that the next best model is actually $d = 0$. This could likely be due to the fact that non-parameterized GPR generally does a good job in finding non-trivial functional dependencies. Adding unnecessary terms to the mean paramterization could actually mess up results more than including no prior model at all. 

It should also be noted that GPR was performed on 3D transfer time datasets where $\textbf{x}_p = ([d_1, d_2, \omega_e])_p^T$ where $\omega_e$ is the vibrational frequency of the single mode present in the system. These results were not fully completed as the required Redfield calculations take exponentially longer as the dimensionality of the desired dataset increases. To increase the parameter space of the metamodel, optimized sampling techniques over these design spaces as mentioned in Section 3 will be necessary to obtain adequate metamodels in a reasonable amount of time. Exploring these routes using the Redfield theory datasets is currently ongoing. 


## 5. References
[1] Y-H. Hwang-Fu, W. Chen, Y-C. Cheng. A coherent modified Redfield theory for excitation energy transfer in molecular aggregates. Journal of Chemical Physics. 447. 46-53. 2015.

[2] R. Jin, W. Chen, T.W. Simpson. Comparative studies of metamodelling techniques under multiple modeling criteria. Structural and Multidisciplinary Optimization. 23. 1-13. 2001.

[3] F. Hase, S. Valleau, E. Pyzer-Knapp, A. Aspuru-Guzik. Machine learning exciton dynamics. Chemical Science. 7. 5139-5147. 2016.

[4] C.E. Rasmussen, K.I. Williams. Gaussian Processes for Machine Learning. The MIT Press. 2006.

[5] P.I. Frazier. A Tutorial on Bayesian Optimization. 2018. 

[6] J.P.C. Kleijnen. Kriging metamodeling in simulation: A review. European Journal of Operational Research. 192. 707-716. 2009.

